<a href="https://colab.research.google.com/github/Raistar22/1JS21AI041/blob/main/Rag-bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -U sentence-transformers
!pip install torch


In [19]:
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]

device = 'cuda' if torch.cuda.is_available() else 'cpu'
embeddings = model.encode(sentences, convert_to_tensor=True, device=device)
similarities = util.pytorch_cos_sim(embeddings, embeddings)

print(similarities.shape)
print(similarities)


torch.Size([4, 4])
tensor([[1.0000, 0.6946, 0.9429, 0.2569],
        [0.6946, 1.0000, 0.6211, 0.2491],
        [0.9429, 0.6211, 1.0000, 0.2106],
        [0.2569, 0.2491, 0.2106, 1.0000]])


In [20]:
!pip install -q pandas faiss-cpu sentence-transformers rank-bm25 rapidfuzz transformers accelerate torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 23.2 MB/s eta 0:00:00


In [21]:
import pandas as pd
import numpy as np
import re
import json
from pathlib import Path

# Embeddings + retrieval
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
from rapidfuzz import process, fuzz

# FAISS
import faiss

# Small local LLM for generation
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ---------- Settings ----------
CSV_PATH = "/B2(Score).csv"   # <-- change to your file path
COMPANY_NAME_COL = None               # set to column name if company names are in a specific column, else None if they are in the index
TEXT_EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # 384-dim, fast
GEN_MODEL = "google/flan-t5-base"     # small, runs on CPU; upgrade to flan-t5-large if you have GPU
TOP_K = 5                             # retrieve top-k docs
RERANK_TOP_K = 3                      # rerank or final select top-k for context


In [30]:
import pandas as pd

# ===== CONFIG =====
CSV_PATH = "/B2(Score).csv"   # ✅ Correct path for Colab
# ===================

# ---- STEP 1: Safe CSV Load ----
encodings_to_try = ['utf-8', 'utf-8-sig', 'latin1', 'cp1252']
df = None

for enc in encodings_to_try:
    try:
        df = pd.read_csv(CSV_PATH, encoding=enc)
        print(f"✅ Loaded CSV using encoding: {enc}")
        break
    except UnicodeDecodeError:
        print(f"❌ Failed with encoding: {enc}")
    except Exception as e:
        print(f"⚠️ Error with encoding {enc}: {e}")

if df is None:
    raise ValueError("❌ Could not read CSV with any known encoding.")

print(f"\n📊 Original shape: {df.shape}")

# ---- STEP 2: Detect and fix transposed structure ----
if df.shape[1] > df.shape[0]:
    print("🔄 Detected transposed format — fixing it now...")
    df = df.transpose().reset_index(drop=False)
    df.rename(columns={'index': 'Criteria'}, inplace=True)

# ---- STEP 3: Use the first row as header ----
df.columns = df.iloc[0]
df = df.drop(df.index[0])

# ---- STEP 4: Rename the first column to 'Company' ----
df.rename(columns={df.columns[0]: "Company"}, inplace=True)

# ---- STEP 5: Clean and remove garbage ----
df = df.dropna(axis=1, how="all")
df = df.dropna(axis=0, how="all")
df.columns = [str(c).strip() for c in df.columns]
df["Company"] = df["Company"].astype(str).str.strip()

# ---- STEP 6: Show summary ----
print(f"✅ Cleaned shape: {df.shape}")
print("\nSample Data:")
display(df.head(5))

# ---- STEP 7: Save clean version ----
clean_path = "/content/cleaned_B2.csv"
df.to_csv(clean_path, index=False, encoding="utf-8")
print(f"\n💾 Cleaned data saved at: {clean_path}")


❌ Failed with encoding: utf-8
❌ Failed with encoding: utf-8-sig
✅ Loaded CSV using encoding: latin1

📊 Original shape: (2161, 1003)
✅ Cleaned shape: (2160, 1003)

Sample Data:


,Company,,1,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,Name of the company - (As of 2024-03-31) Score,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Date of incorporation - (FY 2023-04-01 to 2024...,,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Address of registered office of company - (FY ...,,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Address of corporate office of company - (FY 2...,,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,E mail of the company - (FY 2023-04-01 to 2024...,,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



💾 Cleaned data saved at: /content/cleaned_B2.csv


In [31]:
def row_to_doc(company_name, row: pd.Series) -> dict:
    # Build a readable doc text
    fields = []
    for col, val in row.items():
        if pd.isna(val):
            continue
        txt = str(val).strip()
        if txt:
            fields.append(f"{col}: {txt}")
    doc_text = f"Company: {company_name}\n" + "\n".join(fields)
    return {
        "company": company_name,
        "text": doc_text,
        "metadata": row.to_dict()
    }

docs = [row_to_doc(idx, df.loc[idx]) for idx in df.index]
len(docs), docs[0]["text"][:500]


(2160,
 'Company: 1\nCompany: Name of the company - (As of 2024-03-31) Score\n1: 2')

In [32]:
import pandas as pd

# Reload your cleaned file (or directly from original path)
df = pd.read_csv("/B2(Score).csv", encoding="latin1")

# If your columns look like "BRSR_3M India Limited", etc. — transpose
if df.shape[0] < df.shape[1]:
    print("🔄 Fixing orientation... (criteria as columns, companies as rows)")
    df = df.transpose()

# Make the first row the header
df.columns = df.iloc[0]
df = df.drop(df.index[0])

# Rename index to "Company"
df = df.reset_index().rename(columns={'index': 'Company'})

# Clean whitespace
df.columns = [str(c).strip() for c in df.columns]
df["Company"] = df["Company"].astype(str).str.strip()

print("✅ Final cleaned shape:", df.shape)
print("✅ Sample data:")
display(df.head(5))


✅ Final cleaned shape: (2160, 1004)
✅ Sample data:


,Company,Corporate identity number - (FY 2023-04-01 to 2024-03-31) Score,,1,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,1,Name of the company - (As of 2024-03-31) Score,,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Date of incorporation - (FY 2023-04-01 to 2024...,,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Address of registered office of company - (FY ...,,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Address of corporate office of company - (FY 2...,,5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,E mail of the company - (FY 2023-04-01 to 2024...,,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
def row_to_doc(company_name, row: pd.Series) -> dict:
    # Build a readable doc text
    fields = []
    for col, val in row.items():
        if pd.isna(val):
            continue
        txt = str(val).strip()
        if txt:
            fields.append(f"{col}: {txt}")
    doc_text = f"Company: {company_name}\n" + "\n".join(fields)
    return {
        "company": company_name,
        "text": doc_text,
        "metadata": row.to_dict()
    }

docs = [row_to_doc(idx, df.loc[idx]) for idx in df.index]
len(docs), docs[0]["text"][:500]


(2160,
 'Company: 0\nCompany: 1\nCorporate identity number - (FY 2023-04-01 to 2024-03-31) Score: Name of the company - (As of 2024-03-31) Score\n1: 2')

In [34]:
import pandas as pd

# Load the actual CSV
df = pd.read_csv("/B2(Score).csv", encoding="latin1")

# ---- Step 1: Transpose ----
print("🔄 Transposing so companies become rows...")
df = df.transpose()

# ---- Step 2: Set header row ----
df.columns = df.iloc[0]   # make first row the column names (criteria)
df = df.drop(df.index[0]) # remove header row from data

# ---- Step 3: Create a 'Company' column from the index ----
df = df.reset_index().rename(columns={'index': 'Company'})

# ---- Step 4: Clean up ----
df.columns = [str(c).strip() for c in df.columns]
df["Company"] = df["Company"].astype(str).str.strip()

print("✅ Final cleaned structure:")
print(df.shape)
display(df.head(3))


🔄 Transposing so companies become rows...
✅ Final cleaned structure:
(1002, 2162)


,Company,Corporate identity number - (FY 2023-04-01 to 2024-03-31) Score,Name of the company - (As of 2024-03-31) Score,Date of incorporation - (FY 2023-04-01 to 2024-03-31) Score,Address of registered office of company - (FY 2023-04-01 to 2024-03-31) Score,Address of corporate office of company - (FY 2023-04-01 to 2024-03-31) Score,E mail of the company - (FY 2023-04-01 to 2024-03-31) Score,Telephone of company - (FY 2023-04-01 to 2024-03-31) Score,Website of company - (FY 2023-04-01 to 2024-03-31) Score,Date of start of financial year - (FY 2023-04-01 to 2024-03-31) Score,...,Remarks For Assurance Of Details Of Corrective Action Taken Or Underway On Any Issues Related To Anti Competitive Conduct By The Entity Based On Adverse Orders From Regulatory Authorities Score,Remarks For Assurance Of Details Of Public Policy Positions Advocated By The Entity Score,Remarks For Assurance Of A Preferential Procurement Policy Where Preference To Purchase From Suppliers Comprising Marginalized Or Vulnerable Groups And Its Percentage Of Total Procurement By Value Does Information Technology Constitute Score,Remarks For Assurance Of Weblink Where Information On Products And Services Of The Entity Can Be Accessed Score,Remarks For Assurance Of Steps Taken To Inform And Educate Consumers About Safe And Responsible Usage Of Products And Or Services Score,Remarks For Assurance Of Whether The Entity Display Product Information On The Product Over And Above What Is Mandated As Per Local Laws Score,The Entity Does Not Consider The Principles Material To Its Business Score,The Entity Is Not At A Stage Where Information Technology Is In A Position To Formulate And Implement The Policies On Specified Principles Score,The Entity Does Not Have The Financial Or Human And Technical Resources Available For The Task Score,Information Technology Is Planned To Be Done In The Next Financial Year Score
0,"Criteria (Ex. Human Rights, Climate change etc).",,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Question Sl. No (QID),1,2,3,4,5,6,7,8,9,...,2152,2153,2154,2155,2156,2157,2158,2159,2160,2161
2,BRSR_360 ONE WAM LIMITED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
def row_to_doc(company_name, row: pd.Series) -> dict:
    fields = []
    for col, val in row.items():
        if pd.isna(val):
            continue
        txt = str(val).strip()
        if txt:
            fields.append(f"{col}: {txt}")
    doc_text = f"Company: {company_name}\n" + "\n".join(fields)
    return {
        "company": company_name,
        "text": doc_text,
        "metadata": row.to_dict()
    }

# Build docs correctly
docs = [row_to_doc(row["Company"], row) for _, row in df.iterrows()]
len(docs), docs[0]["text"][:500]


(1002,
 'Company: Criteria (Ex. Human Rights, Climate change etc).\nCompany: Criteria (Ex. Human Rights, Climate change etc).\nNumber of employees or workers including differently abled - Permanent Employees All Genders (FY 2023-04-01 to 2024-03-31) Score: S4\nNumber of employees or workers including differently abled - Permanent Employees Male (FY 2023-04-01 to 2024-03-31) Score: S4\nPercentage of employees or workers including differently abled - Permanent Employees Male (FY 2023-04-01 to 2024-03-31) Score')

In [36]:
import pandas as pd

# Load the actual file again
df = pd.read_csv("/B2(Score).csv", encoding="latin1")

print("📊 Original shape:", df.shape)

# ---- Step 1: Transpose because companies are currently columns ----
print("🔄 Fixing orientation so each company becomes a row...")
df = df.transpose()

# ---- Step 2: Set first row as header (criteria names) ----
df.columns = df.iloc[0]  # make row 0 the header
df = df.drop(df.index[0])  # remove header row from data

# ---- Step 3: Reset index -> becomes Company column ----
df = df.reset_index().rename(columns={'index': 'Company'})

# ---- Step 4: Clean up ----
df.columns = [str(c).strip() for c in df.columns]
df["Company"] = df["Company"].astype(str).str.strip()
df = df.dropna(how="all", axis=1)

print(f"✅ Final shape: {df.shape}")
display(df.head(3))


📊 Original shape: (2161, 1003)
🔄 Fixing orientation so each company becomes a row...
✅ Final shape: (1002, 2162)


,Company,Corporate identity number - (FY 2023-04-01 to 2024-03-31) Score,Name of the company - (As of 2024-03-31) Score,Date of incorporation - (FY 2023-04-01 to 2024-03-31) Score,Address of registered office of company - (FY 2023-04-01 to 2024-03-31) Score,Address of corporate office of company - (FY 2023-04-01 to 2024-03-31) Score,E mail of the company - (FY 2023-04-01 to 2024-03-31) Score,Telephone of company - (FY 2023-04-01 to 2024-03-31) Score,Website of company - (FY 2023-04-01 to 2024-03-31) Score,Date of start of financial year - (FY 2023-04-01 to 2024-03-31) Score,...,Remarks For Assurance Of Details Of Corrective Action Taken Or Underway On Any Issues Related To Anti Competitive Conduct By The Entity Based On Adverse Orders From Regulatory Authorities Score,Remarks For Assurance Of Details Of Public Policy Positions Advocated By The Entity Score,Remarks For Assurance Of A Preferential Procurement Policy Where Preference To Purchase From Suppliers Comprising Marginalized Or Vulnerable Groups And Its Percentage Of Total Procurement By Value Does Information Technology Constitute Score,Remarks For Assurance Of Weblink Where Information On Products And Services Of The Entity Can Be Accessed Score,Remarks For Assurance Of Steps Taken To Inform And Educate Consumers About Safe And Responsible Usage Of Products And Or Services Score,Remarks For Assurance Of Whether The Entity Display Product Information On The Product Over And Above What Is Mandated As Per Local Laws Score,The Entity Does Not Consider The Principles Material To Its Business Score,The Entity Is Not At A Stage Where Information Technology Is In A Position To Formulate And Implement The Policies On Specified Principles Score,The Entity Does Not Have The Financial Or Human And Technical Resources Available For The Task Score,Information Technology Is Planned To Be Done In The Next Financial Year Score
0,"Criteria (Ex. Human Rights, Climate change etc).",,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Question Sl. No (QID),1,2,3,4,5,6,7,8,9,...,2152,2153,2154,2155,2156,2157,2158,2159,2160,2161
2,BRSR_360 ONE WAM LIMITED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
def row_to_doc(company_name, row: pd.Series) -> dict:
    fields = []
    for col, val in row.items():
        if pd.isna(val):
            continue
        txt = str(val).strip()
        if txt:
            fields.append(f"{col}: {txt}")
    doc_text = f"Company: {company_name}\n" + "\n".join(fields)
    return {
        "company": company_name,
        "text": doc_text,
        "metadata": row.to_dict()
    }

# Build the documents
docs = [row_to_doc(row["Company"], row) for _, row in df.iterrows()]
len(docs), docs[0]["text"][:700]


(1002,
 'Company: Criteria (Ex. Human Rights, Climate change etc).\nCompany: Criteria (Ex. Human Rights, Climate change etc).\nNumber of employees or workers including differently abled - Permanent Employees All Genders (FY 2023-04-01 to 2024-03-31) Score: S4\nNumber of employees or workers including differently abled - Permanent Employees Male (FY 2023-04-01 to 2024-03-31) Score: S4\nPercentage of employees or workers including differently abled - Permanent Employees Male (FY 2023-04-01 to 2024-03-31) Score: S4\nNumber of employees or workers including differently abled - Permanent Employees Female (FY 2023-04-01 to 2024-03-31) Score: S4\nPercentage of employees or workers including differently abled - Pe')

In [38]:
import pandas as pd

# Load original file again
df_raw = pd.read_csv("/B2(Score).csv", encoding="latin1")

print("📊 Original shape:", df_raw.shape)

# If there are fewer rows than columns, it's transposed — fix that
if df_raw.shape[0] < df_raw.shape[1]:
    print("🔄 Detected criteria as rows and companies as columns — fixing orientation...")
    df = df_raw.transpose()
else:
    print("✅ Orientation seems correct, continuing...")
    df = df_raw.copy()

# ---- Step 1: Make the first row the header ----
df.columns = df.iloc[0]
df = df.drop(df.index[0])

# ---- Step 2: Reset index to make 'Company' column ----
df = df.reset_index().rename(columns={'index': 'Company'})

# ---- Step 3: Clean column names and company names ----
df.columns = [str(c).strip() for c in df.columns]
df["Company"] = df["Company"].astype(str).str.strip()

# ---- Step 4: Drop empty rows/cols ----
df = df.dropna(axis=0, how="all")
df = df.dropna(axis=1, how="all")

print(f"✅ Final cleaned shape: {df.shape}")
print("🧩 Sample preview:")
display(df.head(3))


📊 Original shape: (2161, 1003)
✅ Orientation seems correct, continuing...
✅ Final cleaned shape: (2160, 1004)
🧩 Sample preview:


,Company,Corporate identity number - (FY 2023-04-01 to 2024-03-31) Score,,1,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,1,Name of the company - (As of 2024-03-31) Score,,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Date of incorporation - (FY 2023-04-01 to 2024...,,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Address of registered office of company - (FY ...,,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df.head(3)["Company"]


,Company
0,1
1,2
2,3


In [40]:
import pandas as pd

# Load your existing structure (rows = criteria, columns = companies)
df = pd.read_csv("/B2(Score).csv", encoding="latin1")

# Clean column names and remove empty ones
df.columns = [str(c).strip() for c in df.columns]
df = df.dropna(how="all", axis=0)
df = df.dropna(how="all", axis=1)

# Extract the first column (criteria names)
criteria = df.iloc[:, 0].astype(str).str.strip()

# Build a document for each company (i.e., each remaining column)
docs = []
for col in df.columns[1:]:  # skip the first criteria column
    company_name = col.strip()
    company_values = df[col].astype(str).tolist()

    # Pair each criterion with its corresponding score/value
    fields = [f"{criteria[i]}: {company_values[i]}" for i in range(len(criteria))]
    doc_text = f"Company: {company_name}\n" + "\n".join(fields)

    docs.append({
        "company": company_name,
        "text": doc_text
    })

print(f"✅ Created {len(docs)} company documents")
print("\n📘 Sample document:\n")
print(docs[0]["text"][:1000])


✅ Created 1002 company documents

📘 Sample document:

Company: Criteria (Ex. Human Rights, Climate change etc).
Corporate identity number - (FY 2023-04-01 to 2024-03-31) Score:  
Name of the company - (As of 2024-03-31) Score:  
Date of incorporation - (FY 2023-04-01 to 2024-03-31) Score:  
Address of registered office of company - (FY 2023-04-01 to 2024-03-31) Score:  
Address of corporate office of company - (FY 2023-04-01 to 2024-03-31) Score:  
E mail of the company - (FY 2023-04-01 to 2024-03-31) Score:  
Telephone of company - (FY 2023-04-01 to 2024-03-31) Score:  
Website of company - (FY 2023-04-01 to 2024-03-31) Score:  
Date of start of financial year - (FY 2023-04-01 to 2024-03-31) Score:  
Date of end of financial year - (FY 2023-04-01 to 2024-03-31) Score:  
Date of start of previous year - (FY 2023-04-01 to 2024-03-31) Score:  
Date of end of previous year - (FY 2023-04-01 to 2024-03-31) Score:  
Date of start of prior to previous year - (FY 2023-04-01 to 2024-03-31) Scor

In [41]:
import pandas as pd

# Load your CSV as is
df = pd.read_csv("/B2(Score).csv", encoding="latin1")

# Clean column names
df.columns = [str(c).strip() for c in df.columns]
df = df.dropna(how="all", axis=0)
df = df.dropna(how="all", axis=1)

# Extract the first column (criteria/question names)
criteria = df.iloc[:, 0].astype(str).str.strip()

# ✅ Skip the first column (it's not a company)
company_columns = df.columns[1:]

# Build a document for each *actual company*
docs = []
for col in company_columns:
    company_name = col.strip()
    company_values = df[col].astype(str).tolist()

    # Pair criteria with corresponding values (if any)
    fields = []
    for i in range(len(criteria)):
        value = company_values[i]
        if value and value.lower() not in ["nan", "none", ""]:
            fields.append(f"{criteria[i]}: {value}")
    doc_text = f"Company: {company_name}\n" + "\n".join(fields)

    docs.append({
        "company": company_name,
        "text": doc_text
    })

print(f"✅ Created {len(docs)} company documents")
print("\n📘 Sample document:\n")
print(docs[0]["text"][:1200])


✅ Created 1002 company documents

📘 Sample document:

Company: Criteria (Ex. Human Rights, Climate change etc).
Corporate identity number - (FY 2023-04-01 to 2024-03-31) Score:  
Name of the company - (As of 2024-03-31) Score:  
Date of incorporation - (FY 2023-04-01 to 2024-03-31) Score:  
Address of registered office of company - (FY 2023-04-01 to 2024-03-31) Score:  
Address of corporate office of company - (FY 2023-04-01 to 2024-03-31) Score:  
E mail of the company - (FY 2023-04-01 to 2024-03-31) Score:  
Telephone of company - (FY 2023-04-01 to 2024-03-31) Score:  
Website of company - (FY 2023-04-01 to 2024-03-31) Score:  
Date of start of financial year - (FY 2023-04-01 to 2024-03-31) Score:  
Date of end of financial year - (FY 2023-04-01 to 2024-03-31) Score:  
Date of start of previous year - (FY 2023-04-01 to 2024-03-31) Score:  
Date of end of previous year - (FY 2023-04-01 to 2024-03-31) Score:  
Date of start of prior to previous year - (FY 2023-04-01 to 2024-03-31) Scor

In [42]:
!pip install openai faiss-cpu tqdm


In [43]:
!pip install sentence-transformers faiss-cpu tqdm


In [44]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from tqdm import tqdm

# Load a lightweight, multilingual model (you can change later)
model = SentenceTransformer("all-MiniLM-L6-v2")

# Create embeddings for your docs
texts = [doc["text"] for doc in docs]
embeddings = np.array([model.encode(t, normalize_embeddings=True) for t in tqdm(texts)], dtype=np.float32)

print("✅ Embeddings created:", embeddings.shape)


100%|██████████| 1002/1002 [03:11<00:00,  5.24it/s]

✅ Embeddings created: (1002, 384)


In [45]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # inner product since we normalized
index.add(embeddings)
print(f"✅ Added {index.ntotal} company vectors")


✅ Added 1002 company vectors


In [46]:
!pip install transformers accelerate


In [47]:
from transformers import pipeline

# Free, small model for QA generation
qa_model = pipeline("text2text-generation", model="google/flan-t5-base")


Device set to use cpu


In [1]:
import torch
torch.cuda.empty_cache()  # free any leftover memory


In [56]:
from transformers import pipeline

qa_model = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    device=-1   # force CPU mode (no CUDA OOM)
)


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [57]:
def query_rag_open(question, top_k=3, max_context_chars=1500):
    # Step 1: Embed the question
    q_emb = model.encode(question, normalize_embeddings=True)
    q_emb = np.array(q_emb, dtype=np.float32).reshape(1, -1)

    # Step 2: Retrieve similar docs
    D, I = index.search(q_emb, top_k)
    retrieved_docs = [docs[i]["text"] for i in I[0]]

    # Step 3: Build context (truncate safely)
    context = "\n\n".join(retrieved_docs)
    if len(context) > max_context_chars:
        context = context[:max_context_chars] + "..."

    # Step 4: Build prompt
    prompt = f"""Answer the question using the company data below.
If the answer is not in the data, say 'Not found'.

Company data:
{context}

Question: {question}
Answer:"""

    # Step 5: Generate
    answer = qa_model(prompt, max_new_tokens=200)[0]['generated_text']
    return answer.strip()


In [58]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss


In [59]:
# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")


In [60]:
# Rebuild embeddings (if you already computed them before, skip and load from disk)
texts = [doc["text"] for doc in docs]
embeddings = np.array([model.encode(t, normalize_embeddings=True) for t in texts], dtype=np.float32)

# Recreate FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)
print(f"✅ FAISS index built with {index.ntotal} company vectors")


✅ FAISS index built with 1002 company vectors


In [68]:
def query_rag_open(question, top_k=3, max_context_chars=900):
    # Step 1: Embed the question
    q_emb = model.encode(question, normalize_embeddings=True)
    q_emb = np.array(q_emb, dtype=np.float32).reshape(1, -1)

    # Step 2: Retrieve similar docs
    D, I = index.search(q_emb, top_k)
    retrieved_docs = [docs[i]["text"] for i in I[0]]

    # Step 3: Truncate long context
    context = "\n\n".join(retrieved_docs)
    if len(context) > max_context_chars:
        context = context[:max_context_chars] + "..."

    # Step 4: Prompt
    prompt = f"""Answer the question using the company data below.
If the answer is not in the data, say 'Not found'.

Company data:
{context}

Question: {question}
Answer:"""

    # Step 5: Use Flan-T5 (already on CPU)
    answer = qa_model(prompt, max_new_tokens=200)[0]['generated_text']
    return answer.strip()


In [69]:
print(query_rag_open("Email of BRSR_3M India Limited"))


Not found


In [70]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from tqdm import tqdm

# Load model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Split each company text into chunks (~1000 characters each)
def chunk_text(text, max_chars=1000):
    chunks = []
    for i in range(0, len(text), max_chars):
        chunk = text[i:i+max_chars]
        chunks.append(chunk)
    return chunks

chunks = []
for doc in docs:
    company = doc["company"]
    for chunk in chunk_text(doc["text"]):
        chunks.append({
            "company": company,
            "text": chunk
        })

print(f"✅ Created {len(chunks)} chunks from {len(docs)} companies.")


✅ Created 16599 chunks from 1002 companies.


In [71]:
texts = [c["text"] for c in chunks]
embeddings = np.array([model.encode(t, normalize_embeddings=True) for t in tqdm(texts)], dtype=np.float32)

# Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

print(f"✅ FAISS index built with {index.ntotal} chunks.")


100%|██████████| 16599/16599 [35:39<00:00,  7.76it/s]

✅ FAISS index built with 16599 chunks.


In [72]:
def query_rag_chunked(question, top_k=5, max_context_chars=1200):
    q_emb = model.encode(question, normalize_embeddings=True)
    q_emb = np.array(q_emb, dtype=np.float32).reshape(1, -1)

    D, I = index.search(q_emb, top_k)
    retrieved_chunks = [chunks[i]["text"] for i in I[0]]

    # Merge smaller contexts
    context = "\n\n".join(retrieved_chunks)
    if len(context) > max_context_chars:
        context = context[:max_context_chars] + "..."

    prompt = f"""
You are a company knowledge assistant.
Use the company data below to answer precisely.

Data:
{context}

Question: {question}
Answer:
"""
    answer = qa_model(prompt, max_new_tokens=200)[0]['generated_text']
    return answer.strip()


In [73]:
print(query_rag_chunked("Date of incorporation of Aarti Drugs Limited"))
print(query_rag_chunked("Human Rights score of 3M India Limited"))
print(query_rag_chunked("Corporate identity number of Aarti Drugs Limited"))


2023-04-01
2093 Water Intensity Per Rupee Of Turnover Score: 2094 Remarks For Assurance Of Whether The Entity Have Any Sites Or Facilities Identified As Designated Consumers Under The Performance Achieve And Trade Scheme Of The Government Of India Score: 2095 Remarks For Assurance Of Mechanisms In Place To Inform Consumers Of Any Risk Of Disruption Or Discontinuation Of Essential Services Score: 2096 Remarks For Assurance Of Whether The Entity Conducted Life Cycle Perspective Or Assessments For Any Of Its Products Or For Its Services Score: 2097 Remarks For Assurance Of Details Of Significant Social Or Environmental Concerns From Production Or Disposal Of Product Or Service With Action Taken To Mitigate The Same Score: 2099 Remarks For Ass Score: 2002 Human Rights Requirements Form Part Of Your Business Agreements And Contracts Is Not Applic
(FY 2023-04-01 to 2024-03-31)


In [74]:
print(query_rag_chunked("Climate change score of BRSR_3M India Limited"))
print(query_rag_chunked("Waste management score of BRSR_Reliance Industries Limited"))
print(query_rag_chunked("Energy consumption of BRSR_Tata Steel Limited"))
print(query_rag_chunked("Greenhouse gas emissions score of BRSR_Adani Power Limited"))


E1-Climate Change (GHG Emissions) Total scope 3 emissions per rupee of turnover - (FY 2022-04-01 to 2023-03-31) Score: E5
0.0 Waste disposed by other disposal operations - (FY 2023-04-01 to 2024-03-31) Score: 100.0 Total waste disposed - (FY 2023-04-01 to 2024-03-31) Score: 100.0 Whether any independent assessment or evaluation or assurance has been carried out by an external agency for waste management - (FY 2023-04-01 to 2024-03-31) Score: 50.0 Whether total scope 3 emissions and its intensity is applicable to the company - (FY 2023-04-01 to 2024-03-31) Score: 100.0 Total scope 3 emissions per rupee of turnover - (FY 2022-04-01 to 2023-03-31) Score: 75.0 Total scope 3 emissions per rupee of turnover - (FY 2022-04-01 to 2023-03-31) Score: 100.0 Waste disposed by other disposal operations - (FY 2023-04
Total energy consumed from renewable and non renewable sources - (FY 2023-04-01 to 2024-03-31) Score: 75.0 Total energy consumed from renewable and non renewable sources - (FY 2023-04-01

In [75]:
print(query_rag_chunked("CSR score of BRSR_Reliance Industries Limited"))
print(query_rag_chunked("Sustainability rating of BRSR_Tata Consultancy Services Limited"))
print(query_rag_chunked("CSR focus area of BRSR_3M India Limited"))


1625 Type Of Assurance For Details Of Corporate Social Responsibility Score: 1626 Whether CSR Is Applicable As Per Section 135 Of Companies Act 2013 Is Assured By Assurer Score: 1627 Assurance Sub Type For Whether Corporate Social Responsibility Is Applicable As Per Section 135 Of Companies Act 2013 Score: 1628 Type Of Assurance For Transparency And Disclosures Compliances Score: 1629 Whether Complaints Or Grievances On Any Of The Principles Under The National Guidelines On Responsible Business Conduct Is Assured By Assurer Score: 1600 Assurance Sub Type For Value Of Shares Paid Up Score: 1601 Whether Name And Contact Details Of The Contact Person In Case Of Any Queries On The Brsr Report Is Assured By Assurer Score: 1600
0.0
CSR focus area of BRSR_3M India Limited


In [81]:
print(query_rag_chunked("Financial year start date of BRSR_Aarti Drugs Limited"))
print(query_rag_chunked("Financial year end date of BRSR_3M India Limited"))
print(query_rag_chunked("Company contact phone number of BRSR_HDFC Bank Limited"))
print(query_rag_chunked("Previous year end date of BRSR_Reliance Industries Limited"))


2020
2024-03-31
0800 008 008
2015-03-27


In [77]:
!pip install sentence-transformers gradio --quiet


In [78]:
from sentence_transformers import CrossEncoder
import numpy as np

# Fast, accurate reranker that works well on CPU
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def retrieve_top_chunks(question, top_k=20):
    """FAISS retrieval: get top_k candidate chunks (texts + companies)."""
    q_emb = model.encode(question, normalize_embeddings=True)
    q_emb = np.array(q_emb, dtype=np.float32).reshape(1, -1)
    D, I = index.search(q_emb, top_k)
    candidates = [chunks[i] for i in I[0]]
    return candidates

def rerank_chunks(question, candidates, top_m=5, batch_size=16):
    """Cross-encode (question, chunk) pairs and return top_m reranked chunks."""
    pairs = [[question, c["text"]] for c in candidates]
    scores = reranker.predict(pairs, batch_size=batch_size)
    # sort by score desc
    ranked = sorted(zip(scores, candidates), key=lambda x: x[0], reverse=True)
    return [c for s, c in ranked[:top_m]]


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [79]:
def query_rag_rerank(question, faiss_k=20, rerank_m=5, max_context_chars=1200, max_new_tokens=200):
    # 1) retrieve many (broad net)
    candidates = retrieve_top_chunks(question, top_k=faiss_k)
    if not candidates:
        return "Not found", []

    # 2) rerank down to the most relevant few
    top_chunks = rerank_chunks(question, candidates, top_m=rerank_m)

    # 3) build compact context
    context = "\n\n".join([c["text"] for c in top_chunks])
    if len(context) > max_context_chars:
        context = context[:max_context_chars] + "..."

    prompt = f"""Answer the question using ONLY the data below. If the answer is not present, reply "Not found".

Data:
{context}

Question: {question}
Answer:"""

    out = qa_model(prompt, max_new_tokens=max_new_tokens)[0]["generated_text"].strip()

    # return answer + sources (company names)
    sources = list({c["company"] for c in top_chunks})
    return out, sources


In [80]:
ans, src = query_rag_rerank("Date of incorporation of BRSR_Aarti Drugs Limited")
print("Answer:", ans)
print("Sources:", src)


Answer: Not found
Sources: ['BRSR_Alembic Pharmaceuticals Limited', 'BRSR_Aarti Drugs Limited', 'BRSR_SMS Pharmaceuticals Limited', 'BRSR_Jubilant Pharmova Limited', 'BRSR_Aarti Pharmalabs Limited']


In [82]:
import gradio as gr

def gradio_answer(question, faiss_k, rerank_m, max_ctx_chars, max_tokens):
    if not question or not question.strip():
        return "Please type a question.", ""
    answer, sources = query_rag_rerank(
        question.strip(),
        faiss_k=int(faiss_k),
        rerank_m=int(rerank_m),
        max_context_chars=int(max_ctx_chars),
        max_new_tokens=int(max_tokens),
    )
    sources_text = "\n".join(sources) if sources else "—"
    return answer, sources_text

with gr.Blocks(title="Company RAG (with Reranking)") as demo:
    gr.Markdown("## 📊 Company RAG Assistant (FAISS + Cross-Encoder Reranking)")
    gr.Markdown("Ask about any company in your dataset. Tip: use exact names like `BRSR_Aarti Drugs Limited`.")

    with gr.Row():
        q = gr.Textbox(label="Your question", placeholder="e.g., Date of incorporation of BRSR_Aarti Drugs Limited", lines=2)
    with gr.Row():
        faiss_k = gr.Slider(5, 50, value=20, step=1, label="FAISS top_k (broad retrieval)")
        rerank_m = gr.Slider(1, 10, value=5, step=1, label="Rerank top_m (keep best)")
    with gr.Row():
        max_ctx_chars = gr.Slider(400, 2000, value=1200, step=50, label="Max context characters")
        max_tokens = gr.Slider(50, 400, value=200, step=10, label="Max new tokens (generation)")

    btn = gr.Button("Ask")
    ans = gr.Textbox(label="Answer")
    src = gr.Textbox(label="Sources (companies used)")

    btn.click(fn=gradio_answer, inputs=[q, faiss_k, rerank_m, max_ctx_chars, max_tokens], outputs=[ans, src])

demo.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>